Libraries and Imports

In [2]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import torch
from torch import Tensor
import torch.nn as nn
from ncps.wirings import AutoNCP
from ncps.torch import LTC
import pytorch_lightning as pl
import torch.utils.data as data

Setting the CUDA float32 precision. Can be changed to "high" if needed.

In [3]:
torch.set_float32_matmul_precision('medium')

Reading the dataset.

In [4]:
train_x = np.loadtxt('dataset/uci-har/train/X_train.txt')
train_y = np.loadtxt('dataset/uci-har/train/y_train.txt').astype(int)

test_x = np.loadtxt('dataset/uci-har/test/X_test.txt')
test_y = np.loadtxt('dataset/uci-har/test/y_test.txt').astype(int)

Transforming the samples and labels

In [5]:
# Normalization on samples

scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.fit_transform(test_x)

In [6]:
# Converting the labels to a label-positional list

conversion_dict = {1: [1, 0, 0, 0, 0, 0], 2: [0, 1, 0, 0, 0, 0], 
                   3: [0, 0, 1, 0, 0, 0], 4: [0, 0, 0, 1, 0, 0], 
                   5: [0, 0, 0, 0, 1, 0], 6: [0, 0, 0, 0, 0, 1]}

conversion_result = []
for e in train_y:
  conversion_result.append(conversion_dict[e])

train_y = np.array(conversion_result)

In [7]:
# Adding a batch dimension on samples data

train_x = np.expand_dims(train_x, axis=0).astype(np.float32)
test_x = np.expand_dims(test_x, axis=0).astype(np.float32)

Defining a possible crop on data and defining values. Can be used to speed up the training.

In [8]:
crop_rate = 0.0
original_len = train_x.shape[1]
final_len = int(original_len * (1-crop_rate))
test_len = test_x.shape[1]

train_x = train_x[:final_len]
train_y = train_y[:final_len]

Transforming data into PyTorch Tensors

In [9]:
# Transforming into PyTorch Tensors

train_x = Tensor(train_x)
train_y = Tensor(train_y.reshape(1, final_len, 6))
test_x = Tensor(test_x)

Should be two lists like [Batch, Data Amount, Sample/Label].

In [10]:
print(train_x.shape)
print(train_y.shape)

torch.Size([1, 7352, 561])
torch.Size([1, 7352, 6])


Defining loaders, models phases and model configuration

In [11]:
dataloader = data.DataLoader(data.TensorDataset(train_x, train_y), batch_size=16, shuffle=True, num_workers=16, persistent_workers=True)

In [12]:
# LightningModule for training a RNNSequence module

class SequenceLearner(pl.LightningModule):
  def __init__(self, model, lr=0.005):
    super().__init__()
    self.model = model
    self.lr = lr

  def training_step(self, batch, batch_idx):
    x, y = batch
    y_hat, _ = self.model.forward(x)
    y_hat = y_hat.view_as(y)
    loss = nn.MSELoss()(y_hat, y)
    self.log("train_loss", loss, prog_bar=True)
    return {"loss": loss}

  def validation_step(self, batch, batch_idx):
    x, y = batch
    y_hat, _ = self.model.forward(x)
    y_hat = y_hat.view_as(y)
    loss = nn.MSELoss()(y_hat, y)

    self.log("val_loss", loss, prog_bar=True)
    return loss

  def test_step(self, batch, batch_idx):
    # Here we just reuse the validation_step for testing
    return self.validation_step(batch, batch_idx)

  def configure_optimizers(self):
    return torch.optim.Adam(self.model.parameters(), lr=self.lr)

In [13]:
out_features = 6 # Output
in_features = 561 # Input

wiring = AutoNCP(32, out_features)

ltc_model = LTC(in_features, wiring, batch_first=True)
learn = SequenceLearner(ltc_model, lr=0.01)
trainer = pl.Trainer(
    log_every_n_steps=1,
    logger=pl.loggers.CSVLogger("log"),
    max_epochs=100,
    gradient_clip_val=1,  # Clip gradient to stabilize training
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


alloc!


Training

In [15]:
trainer.fit(learn, dataloader)

c:\Users\Gusta\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | LTC  | 96.1 K
-------------------------------
77.1 K    Trainable params
19.0 K    Non-trainable params
96.1 K    Total params
0.384     Total estimated model params size (MB)


Epoch 0: 100%|██████████| 1/1 [01:10<00:00,  0.01it/s, v_num=17, train_loss=0.223]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 1/1 [01:10<00:00,  0.01it/s, v_num=17, train_loss=0.223]


Saving (or loading) the trained model

In [ ]:
# Saving the trained model

torch.save(ltc_model.state_dict(), "models/ltc_model.pt")

In [14]:
# Loading a saved model

ltc_model = torch.load('models/ltc_model.pt')

Evaluating the model on test data

In [15]:
# Defining each label meaning

translate_dict = {0: "Walking", 1: "Walking_Upstairs", 2: "Walking_Downstairs", 3: "Sitting", 4: "Standing", 5: "Laying"}

In [20]:
# Using the model to predict data

prediction_results = ltc_model(test_x)[0][0]

In [21]:
hits = 0 # Counter for the amount of correct awnsers

for i in range(0, prediction_results.shape[0]):
  prediction = np.array(prediction_results[i].tolist()).argmax()
  label_response = test_y[i] - 1 # -1 to fit dictionary starting in 0

  if translate_dict[prediction] == translate_dict[label_response]:
    hits += 1

In [22]:
print(f"Model's Accuracy On Test Data: {hits/test_len:.4f}")

Model's Accuracy On Test Data: 0.9250
